Sentiment analysis of movie (IMDB) reviews using dataset provided by the ACL 2011 paper, 
see http://ai.stanford.edu/~amaas/data/sentiment/
This notebook uses neural net models

The plan is to try a variety of neural net based models:
* dense neural network with bag of words
* dense neural network with fixed size input and words mapped to integers
* LSTM
* CNN


In [1]:
import numpy as np
import os
import os.path
import glob
import time

from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import nltk
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras import models
nltk.download('punkt')
import nltk
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /home/jeremie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import wget
import tarfile

# By checking if the directory exists first, we allow people to delete the tarfile without the notebook re-downloading it
if os.path.isdir('aclImdb'):
    print("Dataset directory exists, taking no action")
else:    
    if not os.path.isfile('aclImdb_v1.tar.gz'):
        print("Downloading dataset")
        #!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
        wget.download('http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz')
    else:
        print("Dataset already downloaded")
    
    print("Unpacking dataset")
    #!tar -xf aclImdb_v1.tar.gz 
    tar = tarfile.open("aclImdb_v1.tar.gz")
    tar.extractall()
    tar.close()
    print("Dataset unpacked in aclImdb")

Dataset directory exists, taking no action


In [3]:
# configuration
SAMPLE_SIZE=1000

## Load data

In [4]:
time_beginning_of_notebook = time.time()
positive_file_list = glob.glob(os.path.join('aclImdb/train/pos', "*.txt"))
positive_sample_file_list = positive_file_list[:SAMPLE_SIZE]

negative_file_list = glob.glob(os.path.join('aclImdb/train/neg', "*.txt"))
negative_sample_file_list = negative_file_list[:SAMPLE_SIZE]

import re

# load doc into memory
# regex to clean markup elements 
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r', encoding='utf8')
    # read all text
    text = re.sub('<[^>]*>', ' ', file.read())
    #text = file.read()
    # close the file
    file.close()
    return text

In [5]:
positive_strings = [load_doc(x) for x in positive_sample_file_list]
negative_strings = [load_doc(x) for x in negative_sample_file_list]

positive_tokenized = [word_tokenize(s) for s in positive_strings]
negative_tokenized = [word_tokenize(s) for s in negative_strings]

In [6]:
from collections import Counter
import numpy as np

In [7]:
total_counts = Counter()
all_reviews = positive_tokenized + negative_tokenized
for r in all_reviews:
    for word in r:
        total_counts[word] += 1

In [8]:
vocab = set(total_counts.keys())

In [9]:
vocab_size = len(vocab)
print(vocab_size)

32832


In [10]:
# Create a dictionary of words in the vocabulary mapped to index positions
# (to be used in layer_0)
word2index = {}
for i,word in enumerate(vocab):
    word2index[word] = i

In [11]:
print("ID of 'movie' = {}".format(word2index['movie']))

ID of 'movie' = 9911


In [12]:
def convert_to_bag(review):
    bag = np.zeros(vocab_size)
    for word in review:
        i = word2index[word]
        bag[i]+=1
    return bag

In [13]:
test_bag = convert_to_bag(all_reviews[0])


In [14]:
all_reviews_encoded = [convert_to_bag(x) for x in all_reviews]

In [15]:
all_reviews_encoded[0].shape

(32832,)

In [17]:
# display the map of words to indices
print("word indexes = {}".format(word2index))

word indexes = {'wheel': 0, 'Stiggs': 1, 'minority': 2, 'recite': 3, 'stall': 4, 'romantics': 5, 'fashions': 6, 'Dorday': 7, 'straws': 8, 'MooCow': 9, 'GUY': 10, 'Resembling': 11, 'Butch': 12, 'policemen': 13, ',seriously': 14, '*******spoiler*****': 15, 'Boy¡¨': 16, 'is..a': 17, 'fill': 18, 'Prsoner': 19, 'deposit': 20, 'Lock': 21, 'alibi.These': 22, 'Kasi': 23, 'details': 24, 'White': 25, 'deceit': 26, 'justifiably': 27, 'Kristofferson': 28, 'wept': 29, 'wielded': 30, 'fancy-titled': 31, 'lacks': 32, 'draws': 33, 'ugly': 34, 'heavier': 35, 'Blob': 36, 'Twilight': 37, 'Tamblyn': 38, 'So': 39, 'repent': 40, 'venoms': 41, "Diff'rent": 42, 'EBAY': 43, 'stepsister': 44, 'St.': 45, 'PRETTY': 46, 'per-say': 47, 'animé': 48, 'instructional': 49, 'trout': 50, 'Wow-this': 51, '0/10': 52, 'Capture': 53, 'undeserving': 54, 'Orgazmo': 55, 'Reading': 56, 'lumbering': 57, '2004': 58, 'Hollin': 59, 'goodbye': 60, 'entertaining': 61, 'Furgusson': 62, 'Pufnstuf': 63, 'Corner': 64, 'Lt.Speirs': 65, 'su

In [18]:
import random

positive_labels = []
for i in range(len(positive_tokenized)):
    positive_labels.append('POSITIVE')
negative_labels = []
for i in range(len(negative_tokenized)):
    negative_labels.append('NEGATIVE')
   

In [19]:
labels = positive_labels + negative_labels

num_lables = []

for val in labels:
    if val == 'POSITIVE':
       num_lables.append(1)
    else:
       num_lables.append(0) 
    

In [20]:
reviews_and_labels = list(zip(all_reviews_encoded, num_lables))
random.shuffle(reviews_and_labels)
reviews, labels = zip(*reviews_and_labels)

In [21]:
labels = np.array(labels)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(np.array(reviews), np.array(labels), test_size=0.25)

In [23]:
model = keras.Sequential()
model.add(layers.Dense(50, activation = "relu", input_shape=(vocab_size, )))
model.add(layers.Dense(10, activation = "relu"))
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                1641650   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 1,642,171
Trainable params: 1,642,171
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

In [25]:
results = model.fit(
 X_train, y_train,
 epochs= 15,
 validation_data=(X_test, y_test),
batch_size=500
)

Train on 1500 samples, validate on 500 samples
Epoch 1/15
1500/1500 [==============================] - 2s 1ms/step - loss: 0.6873 - acc: 0.5640 - val_loss: 0.6510 - val_acc: 0.6940
Epoch 2/15
1500/1500 [==============================] - 1s 670us/step - loss: 0.5582 - acc: 0.8967 - val_loss: 0.5779 - val_acc: 0.7780
Epoch 3/15
1500/1500 [==============================] - 1s 755us/step - loss: 0.4152 - acc: 0.9507 - val_loss: 0.5216 - val_acc: 0.7880
Epoch 4/15
1500/1500 [==============================] - 1s 597us/step - loss: 0.3032 - acc: 0.9700 - val_loss: 0.4808 - val_acc: 0.8080
Epoch 5/15
1500/1500 [==============================] - 1s 626us/step - loss: 0.2216 - acc: 0.9853 - val_loss: 0.4484 - val_acc: 0.8140
Epoch 6/15
1500/1500 [==============================] - 1s 625us/step - loss: 0.1609 - acc: 0.9947 - val_loss: 0.4269 - val_acc: 0.8240
Epoch 7/15
1500/1500 [==============================] - 1s 682us/step - loss: 0.1174 - acc: 0.9947 - val_loss: 0.4132 - val_acc: 0.8260
Epo

In [26]:
model = keras.Sequential([
    layers.Dense(256, activation = "relu", input_shape=(vocab_size, )),
    layers.Dense(128, activation = "relu"),
    layers.Dense(1, activation = "sigmoid")
])

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 256)               8405248   
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 8,438,273
Trainable params: 8,438,273
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

In [29]:
results = model.fit(
 X_train, y_train,
 epochs= 10,
 validation_data=(X_test, y_test),
batch_size=500
)

Train on 1500 samples, validate on 500 samples
Epoch 1/10
1500/1500 [==============================] - 2s 2ms/step - loss: 0.6825 - acc: 0.5460 - val_loss: 0.6273 - val_acc: 0.7120
Epoch 2/10
1500/1500 [==============================] - 1s 972us/step - loss: 0.4663 - acc: 0.9227 - val_loss: 0.5208 - val_acc: 0.7880
Epoch 3/10
1500/1500 [==============================] - 1s 973us/step - loss: 0.2647 - acc: 0.9640 - val_loss: 0.4720 - val_acc: 0.8100
Epoch 4/10
1500/1500 [==============================] - 1s 998us/step - loss: 0.1258 - acc: 0.9887 - val_loss: 0.4085 - val_acc: 0.8320
Epoch 5/10
1500/1500 [==============================] - 1s 975us/step - loss: 0.0523 - acc: 0.9967 - val_loss: 0.4127 - val_acc: 0.8380
Epoch 6/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0217 - acc: 0.9987 - val_loss: 0.4242 - val_acc: 0.8400
Epoch 7/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0091 - acc: 1.0000 - val_loss: 0.4657 - val_acc: 0.8240
Epoch 8